In [81]:
import pandas as pd
import re
import datetime
from remove_doublons import MinHashFilter
from tqdm.auto import tqdm
tqdm.pandas()


In [82]:
#filepath = '/home/hoepfl/hiwijob/Laura/crawler/cmpl/serviam_alvarium/scraped_pages_web_archive_org_web_20230321152834_serviam_alvarium_fr.txt'
#filepath = '/home/hoepfl/hiwijob/Laura/crawler/cmpl/terre_et_peuple/scraped_pages_www_terreetpeuple_com.txt'
#filepath = '/home/hoepfl/hiwijob/Laura/crawler/cmpl/arts_enracines/scraped_pages_arts_enracines_fr.txt'
filepath = '/home/hoepfl/hiwijob/Laura/crawler/cmpl/rebellion_hautetfort/scraped_pages_rebellion_hautetfort_com.txt'
date_format = ''


with open(filepath, 'r') as fr: 
    indiv_pages = re.findall(r'<begin-of-url>([\s\S]*?)<end-of-url>', fr.read())

url = []
title = []
date = []
author = []
text = []
for i in indiv_pages: 
    parts = i.split('<separate-parts>\n')
    url.append(parts[0].strip() if parts[0].strip() else None)
    title.append(parts[1].strip() if parts[1].strip() else None)
    date.append(parts[2].strip() if parts[2].strip() else None)
    author.append(parts[3].strip() if parts[3].strip() else None)
    text.append(parts[4].strip() if parts[4].strip() else None)

df = pd.DataFrame({'url': url, 'title': title, 'date': date, 'author': author, 'text': text})

In [83]:
#Modify date

# TODO handle fucked up formats

df['date'] = pd.to_datetime(df['date'], format='mixed', utc=True)
df['date'] = df['date'].replace(pd.Timestamp("1970-01-01 00:59:59+00:00"), pd.NaT)
df = df.sort_values(by='date', ascending=True)
    

In [84]:
# Filter text
DuplicateFilter = MinHashFilter(threshold=0.7)
key = 0 # Necessary for lsh database, has no real function

def clean_up_text(text): 
    re.sub(r'^[^\\\n]{,3}#{4}\s*', '', text) # Transform H4 into normal text
    
    global key
    # TODO: Handle links and images
    split_text = re.split(r'^[^\\\n]{,5}#.*\n*', text, flags=re.MULTILINE)
    for i in range(len(split_text)): 
        block = split_text[i]
        # Delete very short blocks
        if len(block.split()) < 25: 
            split_text[i] = ''
        # Delete longer blocks if duplicates
        if DuplicateFilter.check_new_article(block, verbose=False): # No duplicate
            DuplicateFilter.add_article(block, key)
            key += 1
        else: # Duplicate
            split_text[i] = ''
    
    recombined_text = ''.join(split_text)
    
    if len(recombined_text.split()) < 50: 
        return None
    return recombined_text



In [85]:
dft = df[:20].copy()

In [86]:
dft['text'] = dft['text'].progress_apply(clean_up_text)

100%|██████████| 20/20 [00:00<00:00, 23.80it/s]


In [87]:
print(len(dft['text']))
print(len(dft['text'][dft['text'].isna()]))

20
5


In [88]:
df[:200][['title','text', 'date']]

,title,text,date
339,La revue Rébellion : Rébellion,## 23/10/2005\n\n### [Edito du numéro 13 Juill...,2005-10-23 00:00:00+00:00
345,London's burning ! : Rébellion,## 23/10/2005\n\n### London's burning \!\n\n**...,2005-10-23 00:00:00+00:00
189,front anticapitaliste : Rébellion,## 28/10/2005\n\n### [Pour une ligne de front ...,2005-10-28 00:00:00+00:00
349,Les femmes face au Capitalisme : Rébellion,## 28/10/2005\n\n### Les femmes face au Capita...,2005-10-28 00:00:00+00:00
442,Rébellion n°15 : LES MILLE ET UNE NUITS DU LU...,## 24/11/2005\n\n### Rébellion n°15 : LES MILL...,2005-11-24 00:00:00+00:00
...,...,...,...
150,eurasie : Rébellion,## 24/01/2011\n\n### [Méridien Zéro : Entretie...,2011-01-24 00:00:00+00:00
152,socialisme révolutionnaire : Rébellion,## 02/01/2011\n\n### [Karl Otto Paete sur le n...,2011-02-01 00:00:00+00:00
211,république de weimar : Rébellion,## 02/01/2011\n\n### [Karl Otto Paete sur le n...,2011-02-01 00:00:00+00:00
80,rébellion toulouse : Rébellion,## 02/06/2011\n\n### [Découvrez Rébellion \!](...,2011-02-06 00:00:00+00:00


In [91]:
dft

,url,title,date,author,text
339,http://rebellion.hautetfort.com/archives/categ...,La revue Rébellion : Rébellion,2005-10-23 00:00:00+00:00,None,**> EDITORIAL.**\n\n**Vive les peuples europée...
345,http://rebellion.hautetfort.com/archive/2005/1...,London's burning ! : Rébellion,2005-10-23 00:00:00+00:00,None,None
189,http://rebellion.hautetfort.com/tag/front+anti...,front anticapitaliste : Rébellion,2005-10-28 00:00:00+00:00,None,**\[Rébellion n°12 Mai/Juin 2005\]**\n\nCeux q...
349,http://rebellion.hautetfort.com/archive/2005/1...,Les femmes face au Capitalisme : Rébellion,2005-10-28 00:00:00+00:00,None,**\[Rébellion n°4 Janvier/Février 2004\]**\n\n...
442,http://rebellion.hautetfort.com/archive/2005/1...,Rébellion n°15 : LES MILLE ET UNE NUITS DU LU...,2005-11-24 00:00:00+00:00,None,"A la date où nous écrivons, le couvre-feu vien..."
513,http://rebellion.hautetfort.com/tag/i+want+a+w...,i want a white riot : Rébellion,2006-01-02 00:00:00+00:00,None,**> Edito : Retour à la normal. > Dossier Emeu...
393,http://rebellion.hautetfort.com/archive/2006/1...,Rébellion 21 - Novembre/ Décembre : Rébellion,2006-01-12 00:00:00+00:00,None,**EDITO**\n\nP2 > Prison Break\n\n**ACTUALITE*...
420,http://rebellion.hautetfort.com/archives/categ...,La revue Rébellion : Rébellion,2006-03-27 00:00:00+00:00,None,Chaque exemplaire est disponible contre 3 euro...
484,http://rebellion.hautetfort.com/archive/2006/0...,Le numéro 17 disponible... : Rébellion,2006-03-27 00:00:00+00:00,None,None
344,http://rebellion.hautetfort.com/archives/categ...,Actions militantes : Rébellion,2006-04-06 00:00:00+00:00,None,"Télécharger nôtre tract, collez-le, diffusez-l..."


In [90]:
print(df.iloc[0]['text'])

## 23/10/2005

### [Edito du numéro 13 Juillet/Août 2005](http://rebellion.hautetfort.com/archive/2005/10/23/edito-du-numero-13-juillet-aout-2005.html)

**> EDITORIAL.**

**Vive les peuples européens \!**

Comme nous l’avions souligné dans notre précédent éditorial, la période post-référendaire pourrait laisser apparaître une certain nombre d’opportunités dans la remise en question du consensus que le capital a pu imposer jusqu’ici en France et en Europe. Le peuple français suivi du peuple néerlandais vient de gripper la machine bruxelloise pour notre plus grande satisfaction. Immédiatement, la classe dominante a décidé de geler dans plusieurs pays de l’UE les consultations référendaires qu’elle avait l’intention de conduire au sein d’un scénario qui voulait donner l’impression d’une diversité \(dans certains pays, seules les institutions parlementaires étant consultées\) dans l’acceptation par les peuples de la machination libérale. L’Espagne avait bien ratifié par voie référendaire l